In [14]:
import pandas as pd
import re
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

train_path = '/content/drive/MyDrive/deep_learning/data/train.csv'
test_path = '/content/drive/MyDrive/deep_learning/data/test.csv'

test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)


print(train_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                       id             artist                   name  \
0  6vNcgPTLPyXflD733MmkAO           Warpaint               Undertow   
1  3ugQJQiDJwVKM8szrpAYNa  The Exploding Boy      Cracked / Reasons   
2  5utkTv4TSvgh9fjG386a84           Sanctity    Beneath The Machine   
3  48jyqLwgvWTQcJrlBB27nO        Gipsy Kings            Petite noya   
4  5xmqcLBNjrzOIancAp3bfX        Saxon Shore  This Shameless Moment   

                                album   type  genre  \
0                            The Fool  train   rock   
1                                Four  train   rock   
2                          Once Again  train   rock   
3                    The Very Best Of  train  latin   
4  The Exquisite Death of Saxon Shore  train   rock   

                             subgenres lyrics  
0                      rock---shoegaze    NaN  
1   rock---goth r

In [15]:
train_df = train_df.drop(columns=['id', 'artist', 'name', 'album', 'type'])
test_df = test_df.drop(columns=['id', 'artist', 'name', 'album', 'type'])

train_df = train_df.drop(columns=["subgenres"])
test_df = test_df.drop(columns=["subgenres"])

print(train_df.head())

   genre lyrics
0   rock    NaN
1   rock    NaN
2   rock    NaN
3  latin    NaN
4   rock    NaN


In [16]:
train_df = train_df.dropna(subset=['lyrics'])
test_df = test_df.dropna(subset=['lyrics'])

print(train_df.head())
print(test_df.head())

         genre                                             lyrics
5        blues  By lorries along sir John Rogerson's quay Mr B...
6   electronic  \r\nBlue skies and green fields\r\nI'm thinkin...
8         rock  Fly, fly high my Black Eagle\r\nLet golden thr...
13     hip hop  Why is it ladies only out for money?\r\nBrothe...
18  electronic   Like...\r\n Turn it on, light it up, we gon s...
        genre                                             lyrics
0  electronic  \r\nEveryday After Work\r\nI Go To A Book Stor...
1  electronic  \r\nMy little girl, drive anywhere\r\nDo what ...
2  electronic  \r\nI was trapped under concrete\r\nBuilt from...
3  electronic  \r\nNa, na, na, na, na, na\r\nNa, na, na, na, ...
4  electronic  :\r\nF-A-I-L-U-R-E\r\nWoo\r\n:\r\nWe might as ...


In [17]:
genres = train_df['genre'].unique()

print(genres)
print(len(genres))

['blues' 'electronic' 'rock' 'hip hop' 'funk / soul' 'pop' 'latin' 'jazz'
 'classical']
9


In [18]:
# Converting genres to numerical values
label_dict = {}
for index, possible_label in enumerate(genres):
    label_dict[possible_label] = index
label_dict

print(label_dict)

train_df['genre'] = train_df["genre"].replace(label_dict)
test_df['genre'] = test_df["genre"].replace(label_dict)

print(train_df.head())

{'blues': 0, 'electronic': 1, 'rock': 2, 'hip hop': 3, 'funk / soul': 4, 'pop': 5, 'latin': 6, 'jazz': 7, 'classical': 8}
    genre                                             lyrics
5       0  By lorries along sir John Rogerson's quay Mr B...
6       1  \r\nBlue skies and green fields\r\nI'm thinkin...
8       2  Fly, fly high my Black Eagle\r\nLet golden thr...
13      3  Why is it ladies only out for money?\r\nBrothe...
18      1   Like...\r\n Turn it on, light it up, we gon s...


<ipython-input-18-3512a6508137>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['genre'] = train_df["genre"].replace(label_dict)
<ipython-input-18-3512a6508137>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['genre'] = test_df["genre"].replace(label_dict)


In [19]:
# Remove special characters
def remove_special_characters(text):
    return re.sub(r'[^A-Za-z0-9 ]+', '', text)

train_df['lyrics'] = train_df['lyrics'].apply(remove_special_characters)
test_df['lyrics'] = test_df['lyrics'].apply(remove_special_characters)

print(train_df.head())

    genre                                             lyrics
5       0  By lorries along sir John Rogersons quay Mr Bl...
6       1  Blue skies and green fieldsIm thinking of the ...
8       2  Fly fly high my Black EagleLet golden thread b...
13      3  Why is it ladies only out for moneyBrothers on...
18      1   Like Turn it on light it up we gon set this o...


In [20]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_embeddings = tokenizer(train_df['lyrics'].tolist(), padding=True, truncation=True, return_tensors='pt')
test_embeddings = tokenizer(test_df['lyrics'].tolist(), padding=True, truncation=True, return_tensors='pt')



In [21]:
from torch.utils.data import Dataset, DataLoader
import torch

class LyricsDataset(Dataset):
    def __init__(self, encodings, labels):
        super().__init__()
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


train_labels = train_df["genre"].values
train_dataset = LyricsDataset(train_embeddings, train_labels)
test_labels = test_df["genre"].values
test_dataset = LyricsDataset(test_embeddings, test_labels)

In [ ]:
from transformers import BertForSequenceClassification
from torch import optim
from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
num_epochs = 10
num_classes = 9
learning_rate = 0.001

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

# (Reference for Sulumain)***
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Load the saved model and tokenizer 
# model_path = "path_to_saved_model"
# model = AutoModelForSequenceClassification.from_pretrained(model_path)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

training_arguments = TrainingArguments(
    output_dir='/content/drive/MyDrive/deep_learning/lyrics_results',
    do_eval=True,
    per_device_train_batch_size=8,
    per_gpu_eval_batch_size=8,
    num_train_epochs=num_epochs,
    load_best_model_at_end=True,
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps'
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


<ipython-input-21-83134c748885>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
400,1.773300,1.938927,0.316413,0.199688,0.184883,0.316413
800,1.535800,1.690450,0.394247,0.359374,0.347459,0.394247
1200,1.308300,1.839203,0.389171,0.354987,0.380624,0.389171
1600,0.995400,2.051076,0.385787,0.361824,0.413946,0.385787
2000,0.689000,2.282301,0.367174,0.356193,0.401789,0.367174
2400,0.464000,2.428977,0.390863,0.382944,0.400156,0.390863
2800,0.300400,3.275451,0.367174,0.356367,0.407494,0.367174
3200,0.191100,3.547125,0.402707,0.394430,0.410571,0.402707
3600,0.125300,3.697867,0.390863,0.386480,0.398369,0.390863
4000,0.113800,3.767061,0.387479,0.385788,0.402648,0.387479


Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
<ipython-input-21-83134c748885>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item 

TrainOutput(global_step=4120, training_loss=0.7300755195247317, metrics={'train_runtime': 3534.8508, 'train_samples_per_second': 9.304, 'train_steps_per_second': 1.166, 'total_flos': 8654266498775040.0, 'train_loss': 0.7300755195247317, 'epoch': 10.0})

In [ ]:
model.save_pretrained('/content/drive/MyDrive/deep_learning/lyrics_caption_finetune_model')